#### readme: 
* reference: 
[在Keras模型中使用预训练的词向量](https://keras-cn-docs.readthedocs.io/zh_CN/latest/blog/word_embedding/)
* summary: 
    - method: use pretrained word2vec 百度百科 + CNN
    - TODO: plot **precision and recall curve** 
        - reference: 
            - [medium article](https://towardsdatascience.com/machine-learning-multiclass-classification-with-imbalanced-data-set-29f6a177c1a) + [github code](https://github.com/javaidnabi31/Multi-class-with-imbalanced-dataset-classification/blob/master/20-news-group-classification.ipynb)
            

In [1]:
import os
import numpy as np
import pandas as pd
os.chdir('/Users/liyuan/desktop/CSAir/codes')

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Flatten
from keras.layers import Embedding
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from pandas_ml import ConfusionMatrix

from word2vec_v2 import word2vec

Using TensorFlow backend.


In [8]:
class word2vec():
    def __init__(self):
        self.embeddings_index = {}
        self.MAX_SEQUENCE_LENGTH = 1000
        self.MAX_NUM_WORDS = 20000
        self.EMBEDDING_DIM = 300
        self.VALIDATION_SPLIT = 0.2
        self.all_labeled_data = pd.DataFrame()
        self.labels_index = {}
        self.word_index  = {}
        self.texts = np.array([])
        self.labels = np.array([])
        self.data = np.array([])
        self.X_train = np.array([])
        self.y_train = np.array([])
        self.X_val = np.array([])
        self.y_val = np.array([])
        self.embedding_matrix = np.array([])
        # output is the vectorized predicted output (one-hot encoding of predicted label)
        self.output = np.array([])

    def load_pretrained_vectors(self, file_path):
        f = open(file_path)
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            self.embeddings_index[word] = coefs
        f.close()
        print('Found %s word vectors.' % len(self.embeddings_index))
        return self.embeddings_index 

    def prepare_data(self,data_file_path):
        self.all_labeled_data = pd.read_csv(data_file_path)
        self.texts = self.all_labeled_data.review_tokens.astype('str').values
        self.labels = self.all_labeled_data.label_encoded.values
        
        # get a dictionary that map each original label to its encoded label, e.g., {'中转': 0,...}
        for label in self.all_labeled_data.label.unique().tolist():
            self.labels_index[label] = self.all_labeled_data[self.all_labeled_data['label'] == label]['label_encoded'].unique()[0]

        tokenizer = Tokenizer(nb_words=self.MAX_NUM_WORDS)
        tokenizer.fit_on_texts(self.texts)
        sequences = tokenizer.texts_to_sequences(self.texts)

        self.word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(self.word_index))

        self.data = pad_sequences(sequences, maxlen=self.MAX_SEQUENCE_LENGTH)
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)
        
        # Converts a class vector (integers) to binary class matrix
        self.labels = to_categorical(np.asarray(self.labels))
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)

        # split the data into a training set and a validation set
        self.indices = np.arange(self.data.shape[0])
        np.random.shuffle(self.indices)
        self.data = self.data[self.indices]
        self.labels = self.labels[self.indices]
        nb_validation_samples = int(self.VALIDATION_SPLIT * self.data.shape[0])

        self.X_train = self.data[:-nb_validation_samples]
        self.y_train = self.labels[:-nb_validation_samples]
        self.X_val = self.data[-nb_validation_samples:]
        self.y_val = self.labels[-nb_validation_samples:]
        return  self.X_train, self.y_train, self.X_val, self.y_val
    
    
    def get_embedding_matrix(self):
        # 据得到的字典生成上文所定义的词向量矩阵
        embedding_matrix = np.zeros((len(self.word_index) + 1, self.EMBEDDING_DIM))
        for word, i in self.word_index.items():
            embedding_vector = self.embeddings_index.get(word)
            # updated:
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        return embedding_matrix
    
    def setup_neural_net(self):
        # get word embedding matrix
        self.embedding_matrix = self.get_embedding_matrix()

        # 将这个词向量矩阵加载到Embedding层
        embedding_layer = Embedding(len(self.word_index) + 1,
                                    self.EMBEDDING_DIM,
                                    weights=[self.embedding_matrix],
                                    input_length=self.MAX_SEQUENCE_LENGTH,
                                    trainable=False)
        
        # 使用一个小型的1D卷积解决分类问题
        sequence_input = Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)
        x = Conv1D(128, 5, activation='relu')(embedded_sequences)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = MaxPooling1D(35)(x)  # global max pooling
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(len(self.labels_index), activation='softmax')(x)
        return sequence_input,preds
    
    
    def train_data(self,X_train,y_train,X_val,y_val):
        sequence_input,preds = self.setup_neural_net()
        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        # can change the number of epoch accordingly
        # 7 generates the best performance
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  nb_epoch=7, batch_size=128)  
        
        # evaluate model using model.evaluate()
        scores = model.evaluate(X_val, y_val, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
        # get predicted class label
        self.output = model.predict(X_val)
        predicted_label_list = self.get_pred_label(self.output)
        return predicted_label_list
    
    
    def get_pred_label(self,output):
        '''get predicted class label based on prediction output'''
        predicted_label_list = []
        for i in range(len(output)):
            predicted_label = output[i].argmax(axis=-1)
            predicted_label_list.append(predicted_label)        
        return predicted_label_list
    
    
    def incorporate_pred_label(self):
        '''return prediction results back to df'''
        # indices is a numpy array, need to convert to a list of indices before feed into df to get sub df 
        # recreate df based on the shuffled indices
        indices = self.indices
        all_labeled_data = self.all_labeled_data.iloc[list(self.indices)]
        nb_validation_samples = int(self.VALIDATION_SPLIT * self.data.shape[0])
        print(nb_validation_samples)
        # need to get the indices of the validation data
        train_val_bound = self.data.shape[0] - nb_validation_samples
        # get validation dataset
        val_df = all_labeled_data[train_val_bound:]
        return val_df

    def map_label(self,df,predicted_label_list):
        '''map predicted labels to original class'''
        # print(predicted_label_list[:10])
        label_dct = self.labels_index
        df['pred_label_encodes'] = predicted_label_list
        # get reversed labels_index dictionary
        reversed_label_dct = {}
        for i in range(len(label_dct)):
            reversed_label_dct[list(label_dct.values())[i]] = list(label_dct.keys())[i]

        # map predicted labels
        pred_label = [reversed_label_dct.get(label) for label in predicted_label_list]
        df['pred_label'] = pred_label
        return df
    
    
    def evaluate_performance(self,val_df):
        # evaluate performance
        y_val_true = val_df.label.values
        y_val_pred = val_df.pred_label.values
        self.get_confusion_matrix(y_val_true,y_val_pred) 
        
    
    def get_confusion_matrix(self,y_test,y_pred):
        '''get tp,tn,fp,fn for each class'''
        cm = ConfusionMatrix(y_test, y_pred)
        cm.print_stats()
        
        
    def over_sampling(self):
        '''modeling after over sampling'''
        smote = SMOTE('minority')
        X_train_sm, y_train_sm = smote.fit_sample(self.X_train,self.y_train)
        print(X_train_sm.shape, y_train_sm.shape)
        
        # fit model based on new data set
        predicted_label_list = self.train_data(X_train_sm,y_train_sm,X_val,y_val)
        return predicted_label_list
        

In [9]:
w2v = word2vec()
embeddings_index = w2v.load_pretrained_vectors('../Source_Data/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5')
# X_train, y_train, X_val, y_val = w2v.prepare_data('./res/all_labeled_data_v3.csv')
X_train, y_train, X_val, y_val = w2v.prepare_data('../res/labeled_data_with_without_tk.csv')
predicted_label_list = w2v.train_data(X_train,y_train,X_val,y_val)

Found 635922 word vectors.
Found 4747 unique tokens.
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551,)
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551, 10)


/anaconda3/envs/nlp/lib/python3.6/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '
/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:118: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1241 samples, validate on 310 samples
Epoch 1/7
1241/1241 [==============================] - 39s 31ms/step - loss: 2.1804 - acc: 0.1926 - val_loss: 2.0725 - val_acc: 0.2581
Epoch 2/7
1241/1241 [==============================] - 39s 32ms/step - loss: 1.9744 - acc: 0.3014 - val_loss: 1.8935 - val_acc: 0.2710
Epoch 3/7
1241/1241 [==============================] - 36s 29ms/step - loss: 1.7466 - acc: 0.3658 - val_loss: 1.7035 - val_acc: 0.3968
Epoch 4/7
1241/1241 [==============================] - 33s 26ms/step - loss: 1.6265 - acc: 0.4206 - val_loss: 1.5560 - val_acc: 0.4774
Epoch 5/7
1241/1241 [==============================] - 30s 24ms/step - loss: 1.4117 - acc: 0.5133 - val_loss: 1.4818 - val_acc: 0.4935
Epoch 6/7
1241/1241 [==============================] - 27s 22ms/step - loss: 1.3018 - acc: 0.5673 - val_loss: 1.7916 - val_acc: 0.4581
Epoch 7/7
1241/1241 [==============================] - 30s 24ms/step - loss: 1.1894 - acc: 0.5898 - val_loss: 1.4860 - val_acc: 0.4968
acc: 49.

In [11]:
# check label index
print('label encoding dictionary:', w2v.labels_index)

label encoding dictionary: {'计划': 7, '机上': 5, '中转': 0, '售后': 3, '预订': 9, '设计': 8, '出发': 1, '性能': 4, '行程': 6, '到达': 2}


In [12]:
val_df = w2v.incorporate_pred_label()
val_df = w2v.map_label(val_df,predicted_label_list)
val_df.head()

310


,review,review_tokens,label,label_encoded,pred_label_encodes,pred_label
971,垃圾南航，我选的靠走道位置，结果打印登机牌给我搞到中间位置，挤死人，真后悔一直选南航了,垃圾 南航 我选 走 道 位置 打印 登机牌 搞 位置 挤死 真 后悔 选 南航,出发,1,1,出发
1215,1，我购买的2018-11-12从沈阳到南京的机票为什么自动值机失败，今天因为晚到几分钟，误...,购买 沈阳 南京 机票 自动 值机 失败 晚到 几分钟 误 飞机 几次 客户 电话 咨询 机...,性能,4,1,出发
1158,该登机口也不通知，差评,登机口 通知 差评,出发,1,1,出发
212,登机后在地面等待大约二十分钟时间里客舱里极其闷热，温度大概超过三十度，很多旅客感到不适！经旅...,登机 地面 等待 二十分钟 时间 里 客舱 里 闷热 温度 超过 三十度 旅客 感到 不适 ...,机上,5,5,机上
466,从w舱降为Y舱没有任何补偿，换飞机也不作任何解释。,舱降 舱 补偿 换 飞机 不作 解释,中转,0,5,机上


In [13]:
# evaluate performance
w2v.evaluate_performance(val_df)

Confusion Matrix:

Predicted  中转  出发  到达  售后  性能  机上  行程  计划  设计  预订  __all__
Actual                                                    
中转         17   4   0   0   0   9   0   0   0   0       30
出发          7  31   4   4   2  16   0   0   0   2       66
到达          1  15   8   0   0   4   0   0   0   0       28
售后          0   8   0   8   1   1   0   0   0   8       26
性能          1   7   0   1  11   1   0   0   0  12       33
机上          2   1   1   0   1  53   0   0   0   0       58
行程          0   8   0   0   3   1   0   0   0   3       15
计划          0   2   0   0   0   2   0   0   0   3        7
设计          0   2   0   0   4   2   0   0   0   1        9
预订          0   3   0   0   7   2   0   0   0  26       38
__all__    28  81  13  13  29  91   0   0   0  55      310


Overall Statistics:

Accuracy: 0.4967741935483871
95% CI: (0.4397660648390478, 0.5538445426624075)
No Information Rate: ToDo
P-Value [Acc > NIR]: 4.835326367628054e-14
Kappa: 0.40212889586707373
Mcnemar's Test P-

In [87]:
# implement oversampling
predicted_label_list_os = w2v.over_sampling()

(1472, 1000) (1472, 10)


/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1472 samples, validate on 310 samples
Epoch 1/10
1472/1472 [==============================] - 34s 23ms/step - loss: 2.1866 - acc: 0.1787 - val_loss: 2.1457 - val_acc: 0.1903
Epoch 2/10
1472/1472 [==============================] - 34s 23ms/step - loss: 2.0263 - acc: 0.2480 - val_loss: 2.1061 - val_acc: 0.1774
Epoch 3/10
1472/1472 [==============================] - 31s 21ms/step - loss: 1.8239 - acc: 0.3696 - val_loss: 2.0873 - val_acc: 0.2581
Epoch 4/10
1472/1472 [==============================] - 31s 21ms/step - loss: 1.5745 - acc: 0.4402 - val_loss: 1.8648 - val_acc: 0.3548
Epoch 5/10
1472/1472 [==============================] - 31s 21ms/step - loss: 1.4550 - acc: 0.5034 - val_loss: 1.6944 - val_acc: 0.4484
Epoch 6/10
1472/1472 [==============================] - 31s 21ms/step - loss: 1.2740 - acc: 0.5781 - val_loss: 1.8601 - val_acc: 0.3742
Epoch 7/10
1472/1472 [==============================] - 31s 21ms/step - loss: 1.1756 - acc: 0.6012 - val_loss: 1.6974 - val_acc: 0.4323
E

In [10]:
val_df = w2v.incorporate_pred_label()
val_df_os = w2v.map_label(val_df,predicted_label_list_os)
val_df_os.head()

NameError: name 'incorporate_pred_label' is not defined

In [92]:
# evaluate performance after over sampling
w2v.evaluate_performance(val_df_os)

Confusion Matrix:

Predicted  中转  出发  到达  售后  性能  机上  行程  计划  设计  预订  __all__
Actual                                                    
中转          3   5   0   2   0   3   0  10   0   3       26
出发          5  16  10   4   1   6   0  19   0  14       75
到达          0   2  11   0   0   4   0  17   0   0       34
售后          0   2   0  10   3   1   1   2   0   8       27
性能          0   1   0   0   8   1   0   3   0   9       22
机上          0   4   4   1   2  34   1   9   0   1       56
行程          0   0   0   0   3   0   0   2   0   8       13
计划          0   0   0   0   3   0   0   5   0   1        9
设计          0   0   0   0   3   0   0   0   1   1        5
预订          0   2   0   2   9   0   0   7   0  23       43
__all__     8  32  25  19  32  49   2  74   1  68      310


Overall Statistics:

Accuracy: 0.3580645161290323
95% CI: (0.304666114521708, 0.4142047918295947)
No Information Rate: ToDo
P-Value [Acc > NIR]: 1.6248443439288728e-06
Kappa: 0.27474723724429817
Mcnemar's Test P-

summary: oversampling vs. non-oversampling has very similar result

In [14]:
# checking: y_train, y_val are prediction results already one-hot encoding
y_train
y_val # this is 'y_val_true'

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
# output => probability of each review concerning each class label
output = w2v.output
# construct a result matrix, based on the output matrix (contain probability info)
# pick the index of class(with highest proba) for each review
picked_label_indices = [w2v.output[i].argmax(axis=-1) for i in range(len(output))]
# print('labels picked for each review:', picked_label_indices)

# initialize a res list
res = []
for i in range(len(output)):
    # initialize a list with 0, e.g., [0,0,...0]
    init = [0] * output.shape[1]
    # set the picked label index to be 1, while the rest as 0
    init[picked_label_indices[i]] = 1
    res.append(init)

In [58]:
y_val_pred = np.array(res)
y_val_pred

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [59]:
# plot ROC_AUC
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, y_val_pred)
print('ROC_AUC score:', score)

ROC_AUC score: 0.6479587034113676


In [66]:
# TODO: plot precision and recall curve
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision_recall_curve(y_val, y_val_pred)

ValueError: multilabel-indicator format is not supported